In [4]:
from google.cloud import bigquery
import pandas as pd

churn_df = pd.read_csv("../outputs/churn_predictions.csv")
churn_df['prediction_date'] = pd.Timestamp.today().normalize()

client = bigquery.Client(project="amex-analytics-project")
client.load_table_from_dataframe(
    churn_df,
    "amex-analytics-project.amex_data.churn_predictions"
).result()


c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=amex-analytics-project, location=US, id=5953fa47-1130-4d0d-b17f-7dca92fae444>

In [6]:
from google.cloud import bigquery

client = bigquery.Client(project="amex-analytics-project")

query = """
CREATE OR REPLACE TABLE `amex-analytics-project.amex_data.full_kpi_export` AS
WITH
-- Complaint KPIs
monthly_volume AS (
  SELECT FORMAT_DATE('%Y-%m', DATE(date_received)) AS month, COUNT(*) AS total_complaints
  FROM `amex-analytics-project.amex_data.amex_complaints`
  GROUP BY month
),
top_issues AS (
  SELECT issue, COUNT(*) AS complaint_count
  FROM `amex-analytics-project.amex_data.amex_complaints`
  GROUP BY issue
  ORDER BY complaint_count DESC
  LIMIT 10
),
sentiment_dist AS (
  SELECT sentiment_label, COUNT(*) AS sentiment_count
  FROM `amex-analytics-project.amex_data.amex_complaints`
  GROUP BY sentiment_label
),
channel_counts AS (
  SELECT submitted_via, COUNT(*) AS channel_count
  FROM `amex-analytics-project.amex_data.amex_complaints`
  GROUP BY submitted_via
),
response_dispute AS (
  SELECT timely_response, consumer_disputed, COUNT(*) AS count
  FROM `amex-analytics-project.amex_data.amex_complaints`
  GROUP BY timely_response, consumer_disputed
),

-- Churn KPIs
churn_summary AS (
  SELECT
    COUNT(*) AS total_customers,
    SUM(CASE WHEN actual = 1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN actual = 1 THEN 1 ELSE 0 END) / COUNT(*), 3) AS churn_rate
  FROM `amex-analytics-project.amex_data.churn_predictions`
),
churn_model_perf AS (
  SELECT
    ROUND(SUM(CASE WHEN actual = predicted THEN 1 ELSE 0 END) / COUNT(*), 3) AS accuracy,
    ROUND(SUM(CASE WHEN predicted = 1 AND actual = 1 THEN 1 ELSE 0 END) / 
          SUM(CASE WHEN predicted = 1 THEN 1 ELSE 0 END), 3) AS precision,
    ROUND(SUM(CASE WHEN predicted = 1 AND actual = 1 THEN 1 ELSE 0 END) / 
          SUM(CASE WHEN actual = 1 THEN 1 ELSE 0 END), 3) AS recall
  FROM `amex-analytics-project.amex_data.churn_predictions`
),

-- Escalation KPIs
escalation_summary AS (
  SELECT
    COUNT(*) AS total_cases,
    SUM(CASE WHEN escalated = True THEN 1 ELSE 0 END) AS escalated_cases,
    ROUND(SUM(CASE WHEN escalated = True THEN 1 ELSE 0 END) / COUNT(*), 3) AS escalation_rate
  FROM `amex-analytics-project.amex_data.escalation_predictions`
)

-- Final unified export
SELECT 'monthly_volume' AS metric_type, month AS dim_1, NULL AS dim_2, total_complaints AS value
FROM monthly_volume

UNION ALL
SELECT 'top_issues', issue, NULL, complaint_count
FROM top_issues

UNION ALL
SELECT 'sentiment_distribution', sentiment_label, NULL, sentiment_count
FROM sentiment_dist

UNION ALL
SELECT 'channel_counts', submitted_via, NULL, channel_count
FROM channel_counts

UNION ALL
SELECT 'response_vs_dispute', CAST(timely_response AS STRING), CAST(consumer_disputed AS STRING), count
FROM response_dispute

UNION ALL
SELECT 'churn_kpis', 'churn_rate', NULL, churn_rate
FROM churn_summary

UNION ALL
SELECT 'churn_model_performance', 'accuracy', NULL, accuracy
FROM churn_model_perf

UNION ALL
SELECT 'churn_model_performance', 'precision', NULL, precision
FROM churn_model_perf

UNION ALL
SELECT 'churn_model_performance', 'recall', NULL, recall
FROM churn_model_perf

UNION ALL
SELECT 'escalation_kpis', 'escalation_rate', NULL, escalation_rate
FROM escalation_summary
"""

client.query(query).result()
print("KPI export table created in BigQuery: amex_data.full_kpi_export")


KPI export table created in BigQuery: amex_data.full_kpi_export
